In [1]:
using StarStats

In [7]:
data = @doc compute_distances_and_EEPs

@show data



data = compute*distances*and_EEPs

Function 

# Fields:

  * 



compute*distances*and_EEPs

Function 

# Fields:

  * 


In [ ]:
using Printf
using BenchmarkTools
using Plots
using LaTeXStrings
#plot(legend=false, xflip=true)
plot()
function path_constructor(strings::Vector{String})
    DATA_FOLDER = ENV["STARSTATS_TEST_DATA_FOLDER"]
    return DATA_FOLDER*"/LMC/LMC_0.00_$(strings[1])_1.00.track.gz"
end
masses = [@sprintf("%.3f", x) for x in range(0.9,2.1,step=0.025)]
grid = ModelDataGrid([masses],[:logM])
load_grid(grid,path_constructor,gz_dataframe_loader_with_Teff_and_star_age_fix)
compute_distances_and_EEPs(grid)

for df in grid.dfs
    plot!(df.logTeff, df.logL, 
    legend=false,
    xflip=true, 
    xlabel=L"log (T$_{eff}$/K)",
    ylabel=L"log (L/L$_{\odot}$)")
  
end
plot!()




Test the EEps routines

In [ ]:
plot()
EEPs = grid.EEPs[14,:]
for i in EEPs
    scatter!([grid.dfs[14].logTeff[i]], [grid.dfs[14].logL[i]])
end

track = grid.dfs[14]
xvals = LinRange(0,5,1000)
log_Teff = get_secondary_EEP.(xvals,Ref(track),Ref(:logTeff))
log_L = get_secondary_EEP.(xvals,Ref(track),Ref(:logL))
plot!(log_Teff, log_L, 
seriestype=:scatter,
legend=false,
xflip=true, 
xlabel=L"log (T$_{eff}$/K)",
ylabel=L"log (L/L$_{\odot}$)")

plot!(grid.dfs[14].logTeff, grid.dfs[14].logL, lc=:red)



Load the entire database of grids 

In [8]:
using Printf
using BenchmarkTools
function path_constructor(strings::Vector{String})
    DATA_FOLDER = ENV["STARSTATS_TEST_DATA_FOLDER"]
    return DATA_FOLDER*"/LMC/LMC_$(strings[1])_$(strings[2])_$(strings[3]).track.gz"
end
masses = [@sprintf("%.3f", x) for x in range(0.9,2.1,step=0.025)]
rotation = [@sprintf("%.2f", x) for x in range(0.0,0.9,step=0.1)]
overshoot = [@sprintf("%.2f", x) for x in range(0.5,4.5,step=0.5)]
grid = ModelDataGrid([rotation,masses,overshoot],
    [:rotation,:logM,:overshoot])
#@benchmark 
load_grid(grid,path_constructor,gz_dataframe_loader_with_Teff_and_star_age_fix); 
compute_distances_and_EEPs(grid)

In [ ]:
observable_names = [:logTeff, :logL, :vrot]
observable_values = [4.51974, 4.289877, 70.7195]
observable_errors = [0.2, 0.2, 10]
# potentially parallelize this
grid_likelihood = ModelDataGridLikelihood(grid, observable_names, observable_values, observable_errors);
CI = credible_interval(grid_likelihood, :logM,0.68,1_000_000)

In [ ]:
using Plots
ml= marginalized_likelihood(grid_likelihood,[:logM])
plot(grid.input_values[2], ml)
hline!([CI[2]])
vline!(CI[1])

In [ ]:
using Plots
ml= marginalized_likelihood(grid_likelihood,[:logM, :rotation])
contour(grid.input_values[2], grid.input_values[1], ml', fill=true)

In [ ]:
#for next time 
# track interpolation
# Add comments to package source code

# Turn MESA history into EEP
#   
# Interpolate EEP for single stars
# Incorporate interpolated data into likelihood computation
# Model comparisons

In [ ]:
using Plots
plot(legend=false,
xflip=true, 
xlabel=L"log (T$_{eff}$/K)",
ylabel=L"log (L/L$_{\odot}$)")

xvals = LinRange(0,5, 1000)
plot!(interpolate_grid_quantity.(Ref(grid),Ref([0.13,1.21,1.05]),:logTeff, xvals), interpolate_grid_quantity.(Ref(grid),Ref([0.13,1.21,1.05]),:logL, xvals))
plot!(interpolate_grid_quantity.(Ref(grid),Ref([0.5,1.21,3]),:logTeff, xvals), interpolate_grid_quantity.(Ref(grid),Ref([0.5,1.21,3]),:logL, xvals))

In [ ]:
using Turing, Distributions
@model function star_model(logTeff_obs, logTeff_err, logL_obs, logL_err, vrot_obs, vrot_err, grid)
    x ~ Uniform(0,3)
    logM ~ Uniform(0.9, 1.5)
    rotation ~ Uniform(0,0.9)
    overshoot ~ Uniform(0.5,1.5)
    logTeff = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:logTeff,x)
    logL = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:logL,x)
    vrot = interpolate_grid_quantity(grid,[rotation, logM, overshoot],:vrot,x)

    logTeff_obs ~ Normal(logTeff, logTeff_err)
    logL_obs ~ Normal(logL, logL_err)
    vrot_obs ~ Normal(vrot, vrot_err)
    return logTeff_obs, logL_obs, vrot_obs
end

In [ ]:
using Logging
Logging.disable_logging(Logging.Warn)
## Here needs more analysis for optimization 
chain = sample(star_model(4.51974, 0.2, 4.289877, 0.2, 70.7195, 20, grid), HMCDA(0.15, 0.65), 200;stream=false, progress=true)

In [ ]:
using StatsPlots
plot(chain)

In [ ]:
corner(chain)

testing improving running time

In [ ]:
using BenchmarkTools
mutable struct thing_alt
    a :: Float64
end
mutable struct thing
    a
end
thing1 = thing_alt(2)
thing2 = thing(2)
function add_1_to_thing(thing)
    thing.a = thing.a+1
end

@benchmark add_1_to_thing(thing1)

In [ ]:
@benchmark add_1_to_thing(thing2)

In [ ]:
@code_warntype add_1_to_thing(thing2)

In [ ]:
@code_warntype add_1_to_thing(thing1)

In [ ]:
@benchmark interpolate_grid_quantity(grid,[0.13,1.21,1.05],:logTeff, 1.7)

In [15]:
@code_warntype  interpolate_grid_quantity2(grid,[0.13,1.21,1.05],:logTeff, 1.7)

MethodInstance for interpolate_grid_quantity2(::ModelDataGrid, ::Vector{Float64}, ::Symbol, ::Float64)
  from interpolate_grid_quantity2(grid, grid_parameters, interpolated_quantity, x) in Main at In[10]:1
Arguments
  #self#::Core.Const(interpolate_grid_quantity2)
  grid::ModelDataGrid
  grid_parameters::Vector{Float64}
  interpolated_quantity::Symbol
  x::Float64
Locals
  #13::var"#13#20"
  @_7::Union{Nothing, Tuple{Int64, Int64}}
  @_8::Any
  #8::var"#8#15"
  #7::var"#7#14"
  @_11::Union{Nothing, Tuple{Int64, Int64}}
  @_12::Union{Nothing, Tuple{Int64, Int64}}
  df_index::Vector{Int64}
  yvalues::Any
  grid_values::Matrix{Float64}
  lower_i::Vector{Int64}
  @_17::Union{Nothing, Tuple{Int64, Int64}}
  j::Int64
  i@_19::Int64
  i@_20::Int64
  index@_21::Any
  @_22::Union{Nothing, Tuple{Any, Union{Bool, Tuple}}}
  #9::var"#9#16"
  i@_24::Int64
  #12::var"#12#19"
  #11::var"#11#18"
  #10::var"#10#17"
  xtarget::Float64
  xval2::Float64
  xval1::Float64
  yval2::Any
  yval1::Any
  index@_

In [18]:
function interpolate_grid_quantity2(grid, grid_parameters, interpolated_quantity, x)
    lower_i = zeros(Int, length(grid.input_names))
    grid_values = zeros(length(grid.input_names),2)
    # get values of parameters at edges
    for j in 1:length(grid.input_names)
        for i in 1:(length(grid.input_values[j])-1)
            if grid.input_values[j][i] <= grid_parameters[j] && grid.input_values[j][i+1] >= grid_parameters[j]
                grid_values[j,1] = grid.input_values[j][i]
                grid_values[j,2] = grid.input_values[j][i+1]
                lower_i[j] = i
                break
            end
        end
    end
    for i in lower_i
        if i == 0
            return NaN
        end
    end

    # get value of quantity to interpolate at vertices at a given x
    yvalues = zeros(typeof(x),[2 for i in 1:length(grid.input_names)]...)
    df_index = zeros(Int, length(grid.input_names))
    for index in collect(Base.product([1:2 for i in 1:length(grid.input_names)]...))
        df_index .= lower_i.+index.-1
        if !isassigned(grid.dfs,df_index...)
            return NaN
        end
        yvalues[index...] = get_secondary_EEP(x,grid.dfs[df_index...], interpolated_quantity)
    end

    for i in 1:length(grid.input_names)
        for index in Base.product([1:2 for i in 1:(length(grid.input_names)-i)]...)
            yval1 = yvalues[[1 for j in 1:(i-1)]...,1,index...]
            yval2 = yvalues[[1 for j in 1:(i-1)]...,2,index...]
            xval1 = grid_values[i,1]
            xval2 = grid_values[i,2]
            xtarget = grid_parameters[i]
            yvalues[[1 for j in 1:(i-1)]...,1,index...] = yval1 +(yval2-yval1)*(xtarget-xval1)/(xval2-xval1)
        end
    end

    return yvalues[[1 for i in 1:length(grid.input_names)]...]

end

interpolate_grid_quantity2 (generic function with 1 method)

In [19]:
@benchmark interpolate_grid_quantity2(grid,[0.13,1.21,1.05],:logTeff, 1.7)

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  Tuple{Int64{},[92mInt64[39m,[92mInt64[39m}[0m to an object of type 
[0m  Tuple{Int64{}}
[0mClosest candidates are:
[0m  convert(::Type{T}, ::Tuple{Vararg{Any, N}}) where {N, T<:Tuple} at essentials.jl:332
[0m  convert(::Type{T}, [91m::T[39m) where T<:Tuple at essentials.jl:331
[0m  convert(::Type{T}, [91m::CartesianIndex[39m) where T<:Tuple at multidimensional.jl:131
[0m  ...

In [14]:
using ProfileView 
ProfileView.@profview for i in 1:1000 
                        interpolate_grid_quantity2(grid,[0.13,1.21,1.05],:logTeff, 1.7)
                      end 

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1,000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint